In [ ]:
# Numpy-only analytical solutions for OLS, Weighted LS, and Ridge (no high-level APIs)

import numpy as np

def add_intercept(X: np.ndarray) -> np.ndarray:
    """Prepend an intercept column of ones to X (n x p) -> (n x (p+1))."""
    n = X.shape[0]
    return np.column_stack([np.ones(n), X])

def solve_symmetric_system(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solve A beta = b with a fallback to pseudoinverse if A is singular/ill-conditioned.
    Uses only numpy.linalg.
    """
    try:
        # Prefer solve on the normal equations for speed/clarity
        return np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        # Fallback to Moore–Penrose pseudoinverse if A is singular
        return np.linalg.pinv(A) @ b

def ols_beta(X: np.ndarray, y: np.ndarray, add_const: bool = True) -> np.ndarray:
    """
    Ordinary Least Squares:
        beta = (X^T X)^{-1} X^T y
    """
    if add_const:
        X = add_intercept(X)
    XtX = X.T @ X
    Xty = X.T @ y
    return solve_symmetric_system(XtX, Xty)

def wls_beta(X: np.ndarray, y: np.ndarray, w: np.ndarray, add_const: bool = True) -> np.ndarray:
    """
    Weighted Least Squares with positive weights w (length n).
    Analytical form:
        beta = (X^T W X)^{-1} X^T W y
    Implemented efficiently via multiplying by sqrt(w) to avoid constructing W:
        Let X̃ = diag(sqrt(w)) X,  ỹ = diag(sqrt(w)) y
        beta = (X̃^T X̃)^{-1} X̃^T ỹ
    """
    if add_const:
        X = add_intercept(X)
    sw = np.sqrt(w).reshape(-1, 1)  # (n x 1)
    X_tilde = sw * X                 # (n x p)
    y_tilde = (sw[:, 0] * y)         # (n, )
    XtX = X_tilde.T @ X_tilde
    Xty = X_tilde.T @ y_tilde
    return solve_symmetric_system(XtX, Xty)

def ridge_beta(
    X: np.ndarray,
    y: np.ndarray,
    lam: float,
    add_const: bool = True,
    penalize_intercept: bool = False,
) -> np.ndarray:
    """
    Ridge Regression (L2):
        beta = (X^T X + λ I)^{-1} X^T y
    By default, the intercept is *not* penalized (common practice).
    """
    if add_const:
        X = add_intercept(X)
    XtX = X.T @ X
    Xty = X.T @ y
    p = XtX.shape[0]
    reg = lam * np.eye(p)
    if add_const and not penalize_intercept:
        reg[0, 0] = 0.0  # don't penalize the intercept term
    A = XtX + reg
    return solve_symmetric_system(A, Xty)




# ------------------------
# Minimal demo / validation
# ------------------------
np.random.seed(7)
n, p = 120, 3
X = np.random.randn(n, p)
true_beta = np.array([2.0, -1.5, 0.75, 3.0])  # [intercept, b1, b2, b3]
y = true_beta[0] + X @ true_beta[1:] + 0.3 * np.random.randn(n)

# OLS
beta_ols = ols_beta(X, y, add_const=True)

# WLS: make heteroskedastic weights favoring the first half of observations
w = np.ones(n)
w[: n // 2] = 2.0
beta_wls = wls_beta(X, y, w, add_const=True)

# Ridge: small λ to shrink non-intercept coefficients slightly
beta_ridge = ridge_beta(X, y, lam=1.0, add_const=True, penalize_intercept=False)

print("True beta:  ", true_beta.round(4))
print("OLS beta:   ", np.round(beta_ols, 4))
print("WLS beta:   ", np.round(beta_wls, 4))
print("Ridge beta: ", np.round(beta_ridge, 4))


In [ ]:
import cvxpy as cp
import numpy as np

def simplex_regression_cvxpy(X, y, w=None, lam=0.0, add_const=True, penalize_intercept=False):
    X = np.asarray(X); y = np.asarray(y)
    n, p = X.shape
    b  = cp.Variable(p, nonneg=True)     # slopes on the simplex
    b0 = cp.Variable() if add_const else 0.0

    sw = np.sqrt(w) if w is not None else 1.0
    resid = sw*( (b0 + X@b) - y ) if add_const else sw*(X@b - y)

    reg = lam*cp.sum_squares(b) + (lam*cp.square(b0) if penalize_intercept else 0)
    constraints = [cp.sum(b) == 1]       # simplex
    prob = cp.Problem(cp.Minimize(cp.sum_squares(resid) + reg), constraints)
    prob.solve()
    return np.r_[b0.value, b.value] if add_const else b.value





In [ ]:
# sample usage
np.random.seed(7)
n, p = 120, 3
X = np.random.randn(n, p)
true_beta = np.array([2.0, -1.5, 0.75, 3.0])
y = true_beta[0] + X @ true_beta[1:] + 0.3*np.random.randn(n)

beta = simplex_regression_cvxpy(X, y)   # default: add_const=True
b0, b = beta[0], beta[1:]
print("intercept:", b0)
print("slopes:", b)
print("sum(slopes) =", b.sum(), " min(slopes) =", b.min())


In [ ]:
# with sample weights

w = np.ones(n)
w[: n//2] = 2.0     # weight first half more
beta_w = simplex_regression_cvxpy(X, y, w=w)
print(beta_w)


In [ ]:
# with ridge penalty
beta_ridge = simplex_regression_cvxpy(X, y, lam=1.0, penalize_intercept=False)
print(beta_ridge)


In [ ]:
# no intercept
b_simplex = simplex_regression_cvxpy(X, y, add_const=False)
print("coeffs:", b_simplex, " sum =", b_simplex.sum(), " min =", b_simplex.min())
